In [1]:
import sys
sys.path.append("/home/pranathi/code/screed/")
import glob
import screed
import os
print(screed.__path__)
from multiprocessing import Pool

['/home/pranathi/miniconda3/lib/python3.7/site-packages/screed-1.0.4-py3.7.egg/screed']


In [10]:
K2G_FASTA_PATH = "/mnt/ibm_lg/pranathi/kmer-hashing/golumbeanu2018/kmermaid/k2g_uniprot/extract_coding/"
NO_K2G_FASTA_PATH = "/mnt/ibm_lg/pranathi/kmer-hashing/golumbeanu2018/kmermaid/no_k2g_uniprot/extract_coding/"
K2G_INTERSECT = "/mnt/ibm_lg/pranathi/kmer-hashing/golumbeanu2018/kmermaid/k2g_uniprot/analysis_july9/"

if not os.path.exists(K2G_INTERSECT):
    os.makedirs(K2G_INTERSECT)
else:
    pass

k2g_fastas = glob.glob(os.path.join(K2G_FASTA_PATH, "*_peptides_clean.fasta"))
no_k2g_fastas = glob.glob(os.path.join(NO_K2G_FASTA_PATH, "*_peptides_clean.fasta"))
print(len(k2g_fastas))
print(len(no_k2g_fastas))

print(k2g_fastas[0])


2
1
/mnt/ibm_lg/pranathi/kmer-hashing/golumbeanu2018/kmermaid/k2g_uniprot/extract_coding/SRR5763194_GSM2687512_Mock_12h_Homo_sapiens_RNA-Seq__coding_reads_peptides_clean.fasta


In [11]:
def get_record_names(path):
    print(path)
    db = screed.read_fasta_sequences(path)
    names = db.keys()
    db.close()
    return names

In [12]:
def save_reads_only_in_k2g(index):
    k2g = k2g_fastas[index]
    no_k2g = \
        os.path.join(NO_K2G_FASTA_PATH, os.path.basename(k2g))
    record_names_no_k2g = get_record_names(no_k2g)
    filename = os.path.join(K2G_INTERSECT, os.path.basename(k2g).replace(".fasta", "_intersect.fasta"))
    result_fasta = open(filename, "a")
    print(record_names_no_k2g[:5])
    print(k2g)
    db_k2g = screed.read_fasta_sequences(k2g)
    for name in db_k2g:
        if name not in record_names_no_k2g:
            continue
        result_fasta.write(">{}\n{}\n".format(name, db[name].sequence))  

In [14]:
save_reads_only_in_k2g(1)

/mnt/ibm_lg/pranathi/kmer-hashing/golumbeanu2018/kmermaid/no_k2g_uniprot/extract_coding/SRR5763199_GSM2687517_HIV_6h_Homo_sapiens_RNA-Seq__coding_reads_peptides_clean.fasta


IntegrityError: UNIQUE constraint failed: DICTIONARY_TABLE.name

In [ ]:
p = Pool(12)
p.map(save_reads_only_in_k2g, k2g_fastas)